In [1]:
%load_ext line_profiler

In [268]:
from collections import Counter
from functools import partial

import numpy as np
rng = np.random.default_rng()
np.set_printoptions(suppress=True, linewidth=180, edgeitems=5)
from tqdm.notebook import tqdm, trange
from tqdm.contrib.concurrent import process_map

from scipy.stats import rankdata
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [253]:
# ns = [0,10,10,11,12,13,14,15,16,17]
# ns = [0,5,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
# ns = [0]


students = np.logspace(-1, 1, 21)
display(students.round(4))


# ms = 5
# arrs = np.triu(np.tile(np.arange(32, 37),(5,1)).T) - 20
# display(arrs)
# display(arrs.sum())

# ms = 4
# arrs = np.triu(np.tile(np.arange(31, 35),(4,1)).T) - 20
# display(arrs)
# display(arrs.sum())


# ns = [0,10,10,11,12,13,14,15,16,17]
ns = [0,5,5,6,7,8,9,10,11,12,13,14,15]
ms = 2
arrs = np.array([[1,-10],[.9,.9]])
display(arrs)
display(arrs.sum().round(2))

array([ 0.1   ,  0.1259,  0.1585,  0.1995,  0.2512,  0.3162,  0.3981,  0.5012,  0.631 ,  0.7943,  1.    ,  1.2589,  1.5849,  1.9953,  2.5119,
        3.1623,  3.9811,  5.0119,  6.3096,  7.9433, 10.    ])

array([[  1. , -10. ],
       [  0.9,   0.9]])

-7.2

In [254]:
ranked = True

In [255]:
def make_model(num_nodes=128, num_dense=4):
    
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)
    
    inputs = tf.keras.layers.Input(shape=(ms*ms + ms + 1))
    x = tf.keras.layers.Flatten()(inputs)
    
    for _ in range(num_dense):
        x = tf.keras.layers.Dense(num_nodes, activation='relu')(x)
    
    # output1 = tf.keras.layers.Dense(5, name='Y0')(x)
    # output2 = tf.keras.layers.Dense(5, name='Y1')(x)
    # model = tf.keras.models.Model(inputs=inputs, outputs=[output1, output2])
    
    output1 = tf.keras.layers.Dense(ms, name='Y0')(x)
    output1 = tf.keras.layers.Softmax()(output1)
    model = tf.keras.models.Model(inputs=inputs, outputs=output1)

    
    opt = tf.keras.optimizers.Adam()
    # opt = tf.keras.optimizers.Adam(learning_rate=1)
    # opt = tf.keras.optimizers.SGD()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    return model

def softmax(x, b, axis=-1):
    x = x - np.max(x)
    return np.exp(x/b) / np.exp(x/b).sum(axis=axis, keepdims=True)

def make_x():
    x = arrs
    
    rand = np.random.random(arrs.shape)
    rand = (rand - 0.5)
    x = x + rand*0.01
    
    x = rng.permuted(x, axis=-1)
    return x

def eval_model(model, n = 1, disable=True):
    rewards = []
    branch_rewards = []
    for _ in trange(n, disable=disable):
        x = make_x()
        x_tree = np.concatenate([np.eye(ms+1).astype(int), np.tile(x.flatten(), (ms+1,1))], axis=1)
        y_pred = model(x_tree)
        branch_reward = (x_tree[0,ms+1:].reshape(ms,ms) * y_pred[1:].numpy()).sum(axis=1)
        rewards.append(y_pred[0].numpy())
        # reward = (y_pred[0] * branch_reward).numpy().sum()
        # rewards.append(reward)
        branch_rewards.append(branch_reward)
    return np.array(rewards), np.array(branch_rewards)

#ranked
def make_data(b, n=1, disable=True):
    X, Y = [], []
    for _ in trange(n, disable=disable):
        
        if ranked:
            # Ranked Rewards
            x = make_x()
            r1 = softmax(rankdata(x,axis=1), b)
            r1sum = (r1*x).sum(axis=1)
            r0 = softmax(rankdata(r1sum), b)
        
        else:
            ## Rewards Directly
            x = make_x()
            r1 = softmax(x, b)
            r1sum = (r1*x).sum(axis=1)
            r0 = softmax(r1sum, b)
        
        pos = [0] * (ms+1)
        pos[0] = 1
        x0 = pos + list(x.flatten())
        y0 = rng.choice(np.arange(ms), p=r0)
        
        pos = [0] * (ms+1)
        pos[y0+1] = 1
        x1 = pos + list(x.flatten())
        y1 = rng.choice(np.arange(ms), p=r1[y0])

        X.append(x0)
        Y.append(y0)
        X.append(x1)
        Y.append(y1)
    
    X = np.array(X)
    Y = np.array(Y)
    
    return X,Y

In [256]:
%%time
for b in students:
    r0s = []
    r1s = []
    for _ in trange(1000, disable=True):
        x = make_x()
        
        if ranked:
            r0 = (softmax(rankdata(x,axis=1), b) * x).sum(axis=1)
            r1 = (softmax(rankdata(r0), b) * r0).sum()
            r0s.append(r0)
            r1s.append(r1)
            
        else:
            r0 = (softmax(x, b) * x).sum(axis=1)
            r1 = (softmax(r0, b) * r0).sum()
            r0s.append(r0)
            r1s.append(r1)
            
        
    r0s = np.array(r0s)
    r1s = np.array(r1s)
    print(f"{b:.3f}, {r1s.mean():.3f}, {r0s.mean(axis=0).round(2)}, {softmax(r0s.mean(axis=0), b).round(2)}")


0.100, 0.999, [1.  0.9], [0.73 0.27]
0.126, 0.996, [1.  0.9], [0.68 0.32]
0.158, 0.980, [0.98 0.9 ], [0.62 0.38]
0.200, 0.927, [0.93 0.9 ], [0.53 0.47]
0.251, 0.900, [0.8 0.9], [0.4 0.6]
0.316, 0.887, [0.55 0.9 ], [0.25 0.75]
0.398, 0.847, [0.17 0.9 ], [0.14 0.86]
0.501, 0.756, [-0.32  0.9 ], [0.08 0.92]
0.631, 0.600, [-0.87  0.9 ], [0.06 0.94]
0.794, 0.385, [-1.43  0.9 ], [0.05 0.95]
1.000, 0.132, [-1.96  0.9 ], [0.05 0.95]
1.259, -0.134, [-2.42  0.9 ], [0.07 0.93]
1.585, -0.392, [-2.82  0.9 ], [0.09 0.91]
1.995, -0.628, [-3.15  0.9 ], [0.12 0.88]
2.512, -0.835, [-3.42  0.9 ], [0.15 0.85]
3.162, -1.013, [-3.64  0.9 ], [0.19 0.81]
3.981, -1.162, [-3.81  0.9 ], [0.23 0.77]
5.012, -1.285, [-3.95  0.9 ], [0.28 0.72]
6.310, -1.386, [-4.06  0.9 ], [0.31 0.69]
7.943, -1.468, [-4.15  0.9 ], [0.35 0.65]
10.000, -1.534, [-4.23  0.9 ], [0.37 0.63]
CPU times: user 6.14 s, sys: 38 ms, total: 6.18 s
Wall time: 6.18 s


In [257]:
def train_model(student):
    verbose = False
    if student == students[-1]:
        print(f'student == {student:.3f}')
        verbose = True
    model = make_model()
    Xtrain, Ytrain = make_data(student, 2**16, disable=not verbose)
    model.fit(Xtrain, Ytrain, verbose=verbose)
    reward = eval_model(model, 10000, disable=not verbose)
    model.save(f'models/starting_{student:.3f}.keras')
    return reward

In [258]:
%%time
rewards = process_map(train_model, students, disable=True, max_workers=24)

student == 10.000


  0%|          | 0/65536 [00:00<?, ?it/s]

4096/4096 [==============================] - 39s 9ms/step - loss: 0.6933 - accuracy: 0.5107


  0%|          | 0/10000 [00:00<?, ?it/s]

CPU times: user 1.47 s, sys: 1.49 s, total: 2.96 s
Wall time: 2min 34s


In [259]:
for r, s in zip(rewards, students):
    
    res = r[0]*r[1]
    res = res.sum(axis=1).mean()
    
    print(f"{s:.3f} {res}")

0.100 0.999981437864255
0.126 0.9999610358430817
0.158 0.9027519628910948
0.200 0.9863395877228394
0.251 0.8912512790379505
0.316 0.8970963777436456
0.398 0.6650358300586346
0.501 0.7665234857001156
0.631 0.6912082635774647
0.794 0.3858241399104868
1.000 0.4139545492216933
1.259 -0.2630305767286299
1.585 -0.1543818776446624
1.995 -0.663244560026417
2.512 -0.8927140619874248
3.162 -0.8886593953037042
3.981 -1.2030676130294593
5.012 -1.41034460450296
6.310 -1.73997514878285
7.943 -1.696766888373279
10.000 -1.6839861735288746


In [204]:
# a = np.array([[10,0], [9,6]])
a = arrs
b = 10
display(a)
print()
display(softmax(a, b=b))
print()
display((a * softmax(a, b=b)).sum(axis=1))

array([[ 1. , -2. ],
       [ 0.7,  0.3]])

array([[0.57444252, 0.42555748],
       [0.50999867, 0.49000133]])

array([-0.27667245,  0.50399947])

# Graphs

In [ ]:
import tensorflow as tf
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)
tf.get_logger().setLevel('ERROR')
model2 = tf.keras.models.load_model(f"models/starting_{0.100:.3f}.keras")
%timeit r0, r1 = eval_model(model2, 1)

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super(CustomCallback, self).__init__()

    def on_batch_end(self, batch, logs=None):
        r0, r1 = eval_model(self.model, 10)
        r = (r0*r1).sum(axis=1).mean()
        
        
        try:
            self.model.rewards.append(r)
            self.model.accs.append(logs['accuracy'])
        except:
            self.model.rewards = [r]
            self.model.accs = [logs['accuracy']]

#         # Add the output to the logs
#         if logs is not None:
#             logs['reward'] = r

In [ ]:
models = {}

In [ ]:
%%time

with open('tree.log', 'a') as f:
    for student in students:
        
        print(f'student {student:.3f}', file=f, flush=True)

        models[student] = {}

        for teacher in students[::2][3:10]:

            print(f'teacher {teacher:.3f}', file=f, flush=True)
            models[student][teacher] = []
            for i in range(5):
                print(f'model {i}', file=f, flush=True)
                # model = make_model()
                model = tf.keras.models.load_model(f"models/starting_{student:.3f}.keras")

                opt = tf.keras.optimizers.Adam()
                loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
                model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

                X, Y = currs[f'{teacher:.3f}']
                model.fit(X[:2**17], Y[:2**17], callbacks=[CustomCallback()], verbose=False)
                # model.fit(X[:2**17], Y[:2**17], callbacks=[CustomCallback()])

                models[student][teacher].append(model)


        #         break
        #     break
        # break

In [ ]:
for student, s_models in models.items():
    # print(student, list(s_models))
    df = {}
    for teacher, model in s_models.items():
        kernel_size = 20
        kernel = np.ones(kernel_size) / kernel_size
        rewards = np.array([m.rewards for m in model]).mean(axis=0)
        rewards = np.convolve(rewards, kernel, mode='valid')
        df[f"{teacher:.3f}"] = rewards
    df = pd.DataFrame(df)
    
    print(f'Student {student:.3f}')
    with warnings.catch_warnings():
        warnings.simplefilter(action='ignore', category=FutureWarning)
        ax = sns.lineplot(df[[c for c in df.columns if float(c) >= student]])
        sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        plt.show()
    print()

In [ ]:
df[[c for c in df.columns if float(c) >= 1]]

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)

    ax = sns.lineplot(df)
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
1+1

In [ ]:
ax = sns.lineplot(df[df.columns[4:9]])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
students[::2][4:9]

# Eval Training

In [260]:
def eval_training(student, teacher, verbose=False):
    import tensorflow as tf
    for gpu in tf.config.list_physical_devices('GPU'):
        tf.config.experimental.set_memory_growth(gpu, True)

    
    if student == -1:
        model2 = make_model()
    else:
        model2 = tf.keras.models.load_model(f"models/starting_{student:.3f}.keras")
    # opt = tf.keras.optimizers.SGD()
    opt = tf.keras.optimizers.Adam()
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model2.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    nsum = 0
    
    X, Y = currs[f"{teacher:.3f}"]
    perm = np.random.permutation(len(X))
    X, Y = X[perm], Y[perm]
    
    rewards = []
    for i in ns:
        if i != 0:
            n = 2**i
            # X, Y = make_data(teacher, n, disable=True)
                        
            model2.fit(X[nsum:nsum+n], Y[nsum:nsum+n], verbose=False)
            nsum += n
            
        r0, r1 = eval_model(model2, 10)
        reward = (r0*r1).sum(axis=1).mean()
        rewards.append(reward)
        # print(f"{nsum}    ", end='\r')
    return rewards

In [261]:
def exp(student, verbose=True, n=20):
    # teachers = np.arange(student, 0.40001, 0.02)
    
    teachers = students
    
    t_ = list(teachers) * n
    s_ = [student for _ in t_]
    
    res = process_map(eval_training, s_, t_, max_workers=32, chunksize=1)
    
    return np.array(res)

In [262]:
%%time
ntotal = (2**np.array(ns[1:])).sum()

currs = process_map(make_data, students, [ntotal]*len(students), max_workers=6, chunksize=1)
currs = dict(zip([f"{s:.3f}" for s in students], currs))

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 141 ms, sys: 395 ms, total: 536 ms
Wall time: 1min 41s


In [263]:
%%time
exp(-1, n=1).reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 44.6 ms, sys: 551 ms, total: 596 ms
Wall time: 28.7 s


array([[-0.95, -1.44, -1.47, -0.88, -0.29,  0.09,  0.82,  0.97,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ],
       [-1.35, -1.2 , -1.25, -1.51, -0.01,  0.3 ,  0.59,  0.95,  0.99,  1.  ,  1.  ,  1.  ,  1.  ],
       [-0.79, -1.49, -1.71, -1.38,  0.07, -0.09,  0.86,  0.97,  0.92,  1.  ,  0.99,  1.  ,  0.95],
       [-1.7 , -1.21, -0.93, -1.5 , -0.71, -0.51,  0.17,  0.82,  0.69,  0.96,  0.91,  0.96,  0.97],
       [-2.79, -1.51, -0.58,  0.35, -0.26, -0.56, -0.71,  0.81,  0.84,  0.81,  0.84,  0.84,  0.9 ],
       [-1.99, -0.75,  0.02,  0.49, -0.52, -0.47, -0.32,  0.52,  0.83,  0.81,  0.85,  0.86,  0.89],
       [-3.18, -1.5 , -0.59, -0.03, -0.37, -0.1 , -0.15,  0.67,  0.81,  0.68,  0.82,  0.86,  0.67],
       [-2.03, -1.22, -0.61, -0.45, -0.33, -0.88, -0.67, -0.04,  0.4 ,  0.55, -0.11,  0.67,  0.73],
       [-0.9 , -1.81, -1.2 , -0.31, -0.55, -0.41, -0.75,  0.56,  0.41,  0.35,  0.67,  0.44,  0.63],
       [-2.63, -1.37, -0.8 , -0.18, -0.59, -0.68, -0.72, -0.14,  0.27,  0.22,  0.51,  0.46,  0.27],


In [264]:
%%time
exp(1, n=5).reshape(-1, len(students), len(ns)).mean(axis=0).round(3)

  0%|          | 0/105 [00:00<?, ?it/s]

CPU times: user 231 ms, sys: 592 ms, total: 823 ms
Wall time: 1min 54s


array([[ 0.404,  0.377,  0.363,  0.306,  0.22 ,  0.099,  0.075,  0.741,  0.997,  0.999,  1.   ,  1.   ,  1.   ],
       [ 0.398,  0.386,  0.364,  0.315,  0.217,  0.119,  0.236,  0.74 ,  0.995,  0.999,  0.993,  1.   ,  1.   ],
       [ 0.398,  0.386,  0.364,  0.316,  0.217,  0.117,  0.242,  0.704,  0.985,  0.985,  0.994,  0.99 ,  0.981],
       [ 0.398,  0.386,  0.365,  0.314,  0.217,  0.122,  0.244,  0.697,  0.912,  0.917,  0.904,  0.93 ,  0.933],
       [ 0.398,  0.47 ,  0.514,  0.626,  0.742,  0.855,  0.888,  0.894,  0.896,  0.896,  0.895,  0.89 ,  0.899],
       [ 0.398,  0.451,  0.49 ,  0.586,  0.704,  0.833,  0.864,  0.879,  0.875,  0.88 ,  0.886,  0.876,  0.878],
       [ 0.398,  0.471,  0.514,  0.609,  0.76 ,  0.843,  0.852,  0.851,  0.853,  0.851,  0.846,  0.839,  0.834],
       [ 0.398,  0.445,  0.481,  0.579,  0.727,  0.807,  0.773,  0.714,  0.762,  0.764,  0.745,  0.706,  0.756],
       [ 0.398,  0.344,  0.329,  0.394,  0.488,  0.567,  0.559,  0.552,  0.642,  0.563,  0.558, 

In [265]:
%%time
exp(10, n=5).reshape(-1, len(students), len(ns)).mean(axis=0).round(2)

  0%|          | 0/105 [00:00<?, ?it/s]

CPU times: user 144 ms, sys: 687 ms, total: 831 ms
Wall time: 1min 54s


array([[-1.7 , -1.71, -1.72, -1.68, -1.71, -1.73, -1.78, -1.89, -2.14, -2.58, -2.4 , -2.83, -2.49],
       [-1.71, -1.69, -1.7 , -1.69, -1.71, -1.73, -1.79, -1.9 , -2.15, -2.5 , -2.48, -2.89, -2.41],
       [-1.71, -1.69, -1.7 , -1.69, -1.71, -1.73, -1.79, -1.9 , -2.15, -2.5 , -2.48, -2.88, -2.41],
       [-1.71, -1.69, -1.7 , -1.69, -1.71, -1.73, -1.79, -1.89, -2.15, -2.5 , -2.47, -2.89, -2.4 ],
       [-1.71, -1.68, -1.69, -1.66, -1.65, -1.62, -1.64, -1.31, -0.93, -0.64, -0.69, -0.57, -0.74],
       [-1.71, -1.68, -1.69, -1.66, -1.65, -1.62, -1.64, -1.32, -0.94, -0.69, -0.76, -0.62, -0.78],
       [-1.71, -1.68, -1.69, -1.66, -1.65, -1.62, -1.64, -1.32, -0.96, -0.77, -0.85, -0.72, -0.89],
       [-1.71, -1.68, -1.69, -1.66, -1.65, -1.63, -1.64, -1.34, -1.01, -0.85, -0.99, -0.84, -1.  ],
       [-1.71, -1.69, -1.69, -1.67, -1.66, -1.63, -1.65, -1.36, -1.08, -1.01, -1.12, -0.97, -1.14],
       [-1.71, -1.68, -1.69, -1.66, -1.65, -1.63, -1.65, -1.38, -1.16, -1.14, -1.23, -1.09, -1.24],
